# ECE 590, Fall 2019 
## Problem Set 4
* ### __Important :__  You are not allowed to use built-in optimizers from Pytorch or any other Python Deep Learning environment. 

## Full name: Ana B. Barcenas


### Problem 1 (First-order Optimization Methods)

In [165]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

# Download and prepare the MNIST data set 
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor())


In [170]:
# Let's extract target and features of the dataset and store them as tensors
N_train = 60000
N_test = 10000

train_X = train_dataset.data.reshape(N_train, 28*28).float()
train_y = torch.zeros(N_train, 10).scatter_(1, train_dataset.targets.unsqueeze(1), 1)
test_X = test_dataset.data.reshape(N_test, 28*28).float()
test_y = torch.zeros(N_test, 10).scatter_(1, test_dataset.targets.unsqueeze(1), 1)

In [191]:
class logistic_MNIST():
    
    def __init__(self, optimizer, eta=0.0001, mom_beta=0.9, nag_beta=0.95, lam=0.1, 
                 batch_size=500, n_iters=5, verbose=0):
        
        self.train_X = train_dataset.data.reshape(N_train, 28*28).float()
        self.train_y = torch.zeros(N_train, 10).scatter_(1, train_dataset.targets.unsqueeze(1), 1)
        self.test_X = test_dataset.data.reshape(N_test, 28*28).float()
        self.test_y = torch.zeros(N_test, 10).scatter_(1, test_dataset.targets.unsqueeze(1), 1)
        self.train_target = train_dataset.targets
        self.test_target = test_dataset.targets
        
        self.optimizer = optimizer
        self.eta = eta
        self.lam = lam
        self.batch_size = batch_size
        self.n_iters = n_iters
        self.verbose = verbose
        self.W = torch.randn(28*28, 10, requires_grad=True)
        
        if optimizer == "Momentum":
            self.mom_beta = mom_beta
            self.pre_momentum = torch.zeros_like(self.W)
            
        if optimizer == "NAG":
            self.nag_beta = nag_beta
            self.pre_nag_beta = torch.zeros_like(self.W)
        pass
        
        
    def fit(self):
        for k in range(self.n_iters):
            b_k = np.random.choice(N_train, self.batch_size, replace=True)
            eval("self."+self.optimizer+"(b_k)")
            if self.verbose == 1:
                self.print_result(k)
        return self
    
    
    def predict(self):
        y_pred = self.train_X.mm(self.W)
        test_acc = (self.test_target == self.test_X.mm(self.W).max(dim=1).indices).sum().item() / N_test
        return y_pred, test_acc
    
    
    def SGD(self, b_k):
        for k in range(len(b_k)):
            i_k = np.random.choice(b_k, 1).item()
            y_pred = self.train_X[i_k].unsqueeze(0).mm(self.W)
            
            loss1 = (-self.train_y[i_k] * torch.log_softmax(y_pred, dim=1).squeeze()).sum()
            loss2 = self.lam * self.W.norm(p=2)
            total_loss = loss1 + loss2
            
            total_loss.backward()
            with torch.no_grad():
                self.W -= self.eta * self.W.grad
                self.W.grad.zero_()
        pass
    
    
    def Momentum(self, b_k):
        for k in range(len(b_k)):
            i_k = np.random.choice(b_k, 1).item()
            y_pred = self.train_X[i_k].unsqueeze(0).mm(self.W)
            loss1 = (-self.train_y[i_k] * torch.log_softmax(y_pred, dim=1).squeeze()).sum()
            loss2 = self.lam * self.W.norm(p=2)
            loss = loss1 + loss2
            loss.backward()
            with torch.no_grad():
                self.pre_momentum = self.mom_beta * self.pre_momentum + self.W.grad
                self.W -= self.eta * self.pre_momentum
                self.W.grad.zero_()
        pass
    
              


    def results(self, k=None):
        train_pred = self.train_X.mm(self.W)
        train_accuracy = (self.train_target == train_pred.max(dim=1).indices).sum().item() / N_train
        test_pred = self.test_X.mm(self.W)
        test_accuracy = (self.test_target == test_pred.max(dim=1).indices).sum().item() / N_test
        if k:
            print("Train Accuracy: %.2f%%" % (100*train_accuracy))
            print("Test Accuracy: %.2f%%" % (100*test_accuracy))
        else:
            print("Train Accuracy: %.2f%%   Test Accuracy: %.2f%%" % (100*train_accuracy, 100*test_accuracy))
        pass
    

In [195]:
# Let's get some results using the methods coded above:

batch_size=500
n_iters=120

#### SGD ####
print('Stochastic Gradient Descent:')
sgd = logistic_MNIST(optimizer="SGD", batch_size=batch_size, n_iters=n_iters).fit()
sgd.results()

#### Momentum ####
print('Momentum:')
momentum = logistic_MNIST(optimizer="Momentum", batch_size=batch_size, n_iters=n_iters).fit()
momentum.results()



Stochastic Gradient Descent:
Train Accuracy: 86.63%   Test Accuracy: 86.81%
Momentum:
Train Accuracy: 87.08%   Test Accuracy: 87.30%


### Problem 2 (Newton’s Method for Non-linear Optimization)

In [155]:
# Objective function

import numpy as np
import math

def obj_func(x_1, x_2):
    ''' This function takes x_1 and x_2 as single scalars
    and returns the output of substituting the values into the objective function'''
    output = math.exp(x_1 + (3*x_2) - 0.1) + math.exp(x_1 - (3*x_2) - 0.1) + math.exp(- x_1 - 0.1)
    return output


##### HESSIAN MATRIX ######

def hessian(x_1, x_2):
    ''' This function takes x_1 and x_2 as single scalars
    and computes its hessian matrix'''
    d_x1_x1 = math.exp(x_1 + (3*x_2) - 0.1) + math.exp(x_1 - (3*x_2) - 0.1) + math.exp(- x_1 - 0.1)
    d_x1_x2 = 3*math.exp(x_1 + (3*x_2) - 0.1) - 3*math.exp(x_1 - (3*x_2) - 0.1)
    d_x2_x1 = 3*math.exp(x_1 + (3*x_2) - 0.1) - 3*math.exp(x_1 - (3*x_2) - 0.1)
    d_x2_x2 = 9*math.exp(x_1 + (3*x_2) - 0.1) + 9*math.exp(x_1 - (3*x_2) - 0.1)   
    hess = np.array([[d_x1_x1, d_x1_x2],[d_x2_x1, d_x2_x2]])
    return hess

def differentiation(x_1, x_2):
    ''' This function takes x_1 and x_2 as single scalars
    and computes its hessian matrix'''
    d_x1 = math.exp(x_1 + (3*x_2) - 0.1) + math.exp(x_1 - (3*x_2) - 0.1) - math.exp(- x_1 - 0.1)
    d_x2 = 3*math.exp(x_1 + (3*x_2) - 0.1) - 3*math.exp(x_1 - (3*x_2) - 0.1)  
    diff = np.array([d_x1, d_x2])
    return diff


print('Hessian matrix for x_1 = 1 and x_2 = 1:')
print(hessian(1,1))



Hessian matrix for x_1 = 1 and x_2 = 1:
[[ 49.85777662 147.83997803]
 [147.83997803 445.7241498 ]]


In [153]:
##### A) ######

def newton_meth(step_size, x_1, x_2):
    X = np.array([x_1, x_2])
    X = X - step_size*np.matmul(np.linalg.inv(hessian(x_1, x_2)),differentiation(x_1, x_2))
    return X

# Define the starting point X
x_1 = 1
x_2 = 1
X = np.array([x_1, x_2])

# Let's run 10,000 iterations and set up a stopping criteria comparing the change in losses after each iterarion
for t in range(1,10000):
    # Step size
    step_size = 0.01
    loss = obj_func(X[0], X[1])
    X = newton_meth(step_size, X[0], X[1])
    # Stopping criteria
    if loss - obj_func(X[0], X[1]) < 0.001:
        loss = obj_func(X[0], X[1])
        break
print(loss)


2.6123757974367656


In [156]:
##### B) ######

## I will only update the values of the diagonal and keeping the other values in the Hessian as zeros:

for t in range(1,10000):
    hess = hessian(X[0], X[1])
    hess[0,1] = 0
    hess[1,0] = 0
    
    # Step size
    step_size = 0.01
    loss = obj_func(X[0], X[1])
    X = X - step_size*np.matmul(np.linalg.inv(hess),differentiation(X[0], X[1]))
    # Stopping criteria
    if loss - obj_func(X[0], X[1]) < 0.001:
        loss = obj_func(X[0], X[1])
        break
print(loss)

2.6049802357182354


### Problem 3: Logistic Regression using Newton’s Method

In [225]:
"""
Breast cancer dataset preparation
"""

import pandas as pd
import numpy as np
import math 
from sklearn.model_selection import train_test_split

# read csv file 
df = pd.read_csv('breast_cancer.csv')

# extract the 'diagnosis' column as your targets 
# and convert the entries of targets to 0/1 
targets = pd.get_dummies(df.diagnosis).M

# extract your features data
data = df[[ 'radius_mean', 'texture_mean', 'smoothness_mean',
       'compactness_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'smoothness_se', 'compactness_se',
        'symmetry_se', 'fractal_dimension_se']]

# train/test split 
X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.3, random_state=53)

In [226]:
# Let's define the sigmoid function
def sigmoid(a):
    return 1/(1+np.exp(-a))


# Compute the differentiation per row
def differentiation_row(X,y,w):
    sigm = sigmoid(np.matmul(np.transpose(w),X))
    row_1o = (sigm - y)*X
    row_2o = sigm*(1-sigm)*np.matmul(X.reshape(-1,1),np.transpose(X).reshape(1,-1))    
    return row_1o, row_2o

# Compute differentiation per term
def differentiation(X,y,w):
    first_o = np.zeros(X.shape[1])
    second_o = np.zeros((X.shape[1],X.shape[1]))  
    for i in range(X.shape[0]):
        row_1o, row_2o = differentiation_row(np.array(X.iloc[i]),np.array(y.iloc[i]),w)
        first_o_ = first_o + row_1o
        second_o_ = second_o + row_2o
    return first_o_, second_o_

# Output of the objective function
def obj_func(X,y,w):
    output = 0
    for i in range(X.shape[0]):
        sigm = sigmoid(np.matmul(np.transpose(w),np.array(X.iloc[i])))
        row_val = (- np.array(y.iloc[i]) * np.log(sigm)) - ((1-np.array(y.iloc[i])) * np.log(1-sigm))
        output += row_val
    return output

In [228]:
#### A) Hessian Matrix ####

# Define random weights
w = np.random.rand(X_train.shape[1])

for i in range(1,1000):
    output, hess = differentiation(X_train,y_train,w)
    step_size = 0.01
    
    loss = obj_func(X_train,y_train,w)
    w = w - step_size*np.matmul(output,np.linalg.inv(hm)) 
    #stopping criteria
    if loss - obj_func(X_train,y_train,w) < 0.01:
        loss = obj_func(X_train,y_train,w)
        break
        
print("Minimized loss (Hessian):",loss)

/Users/anabelen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/anabelen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/Users/anabelen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


Minimized loss (Hessian): nan


In [229]:
#### A) Hessian diagonal approximation ####

# Define random weights
w = np.random.rand(X_train.shape[1])

for i in range(1,1000):
    output, hess = differentiation(X_train,y_train,w)
    hess[0,1] = 0
    hess[1,0] = 0
    
    step_size = 0.01
    loss = obj_func(X_train,y_train,w)
    w = w - step_size*np.matmul(output,np.linalg.inv(hm)) 
    #stopping criteria
    if loss - obj_func(X_train,y_train,w) < 0.01:
        loss = obj_func(X_train,y_train,w)
        break
        
print("Minimized loss (Hessian diagonal approximation):",loss)

/Users/anabelen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/anabelen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/Users/anabelen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


Minimized loss (Hessian diagonal approximation): nan
